<a href="https://colab.research.google.com/github/iron59zug/YFapi/blob/master/api_iex_cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
# calling to API by a request w/ interpolated values
# IEX Cloud API not free of charge
# video course from freeCodeCamp (4 dec 2020) https://www.youtube.com/watch?v=xfzGZB4HhEE
# model file

import math
import numpy as np # coded in C, example of Python as glue for other faster languages
import pandas as pd # to treat tabular data eg from csv
import requests # for http communication
# not working in colab? import xlsxwriter # save output in excel

In [53]:
a1 = 'MSFT' # 1975
a2 = 'APPL' # 1976
a3 = 'AMZN' # 1994
a4 = 'GOOGL' # 1998; Alphabet's A share has 1 vote; C share "GOOG" has no vote
a5 = 'FB' # 2004

ticker = a1 # choose one asset name that will pass to ticker, or:
tickers = [a1, a2, a3, a4, a5] # use a list of tickers
tstring = ','.join(tickers) # serialize list into string

tstring

'MSFT,APPL,AMZN,GOOGL,FB'

In [54]:
endpoint = 'chart'
period = '1y'
KEY = 'Tpk_059b97af715d417d9f49f50b51b1c448' # free IEX Cloud API key for testing
''' for API key stored in key.py file
    (IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448')
from key import IEX_CLOUD_API_TOKEN
'''

# request = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={string}&types={endpoint}&range={period}&token={KEY}' # non-free api
request = f'https://sandbox.iexapis.com/stable/stock/{a1}/quote?token={KEY}' # free, not real data #fstring
df = requests.get(request)# get/post/put/delete
print (df.status_code == 200) # check url if 404

True


In [55]:
df = requests.get(request).json()
df

{'avgTotalVolume': 26860888,
 'calculationPrice': 'close',
 'change': 3.45,
 'changePercent': 0.0154,
 'close': 234.32,
 'closeSource': 'flfoicia',
 'closeTime': 1646574116382,
 'companyName': 'Microsoft Corporation',
 'delayedPrice': 240.22,
 'delayedPriceTime': 1671328033007,
 'extendedChange': -0.39,
 'extendedChangePercent': -0.00168,
 'extendedPrice': 238,
 'extendedPriceTime': 1639220430625,
 'high': 241.06,
 'highSource': 'yil emrea1eidtd ecnp u5',
 'highTime': 1645575037952,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 237.4,
 'iexCloseTime': 1615988816270,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 238.5,
 'iexOpenTime': 1655585219808,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'isUSMarketOpen': False,
 'lastTradeTime': 1678286058647,
 'latestPrice': 236.24,
 'latestSource': 'Close',
 'latestTime': 'February 26, 2021',
 'latestUpdate': 1652078657559,
 'latestVolume': 38726

In [56]:
close = df['close']
mc = df['marketCap']
close,mc

(234.32, 1814243813871)

In [57]:
ct = ['ticker', 'close price', 'market cap', 'buy qty']
final_df = pd.DataFrame([[1,2,3,4]], columns = ct)
final_df

,ticker,close price,market cap,buy qty
0,1,2,3,4


In [58]:
final_df.append(
    pd.Series(
        [ticker,
         close,
         mc,
         'n/a'
        ],
    index = ct
    ),
    ignore_index=True # necessary whenever use append???
)

,ticker,close price,market cap,buy qty
0,1,2.00,3,4
1,MSFT,234.32,1814243813871,n/a


In [65]:
# using dynamic input from certain API would be ideal; using static list for now
from google.colab import drive
drive.mount('/content/drive')

# load static ticker list
tlist = pd.read_csv('/content/drive/MyDrive/sp_500_stocks.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
final_df = pd.DataFrame(columns = ct)
for ticker in tlist['Ticker'][:5]: # check first 5 tickers only
  request = f'https://sandbox.iexapis.com/stable/stock/{ticker}/quote?token={KEY}' 
  df = requests.get(request).json() # single api request via http = slow!
  final_df = final_df.append(
      pd.Series(
          [ticker,
           df['latestPrice'],
           df['marketCap'],
           'n/a'           
          ],
          index = ct),
      ignore_index = True
  )

#del fdf
final_df

,ticker,close price,market cap,buy qty
0,A,125.76,38251553209,n/a
1,AAL,20.98,13668737849,n/a
2,AAP,166.00,10782434453,n/a
3,AAPL,121.42,2129023243737,n/a
4,ABBV,109.09,194443349499,n/a


In [73]:
def chunks(lst, n): # function to divide ticker list for multiple api request
  for i in range(0, len(lst),n):
    yield lst[i:i + n]

tgroups = list(chunks(tlist['Ticker'], 101))
for i in range(0, len(tgroups)):
  print(tgroups[i])
  

0          A
1        AAL
2        AAP
3       AAPL
4       ABBV
       ...  
96        CL
97       CLX
98       CMA
99     CMCSA
100      CME
Name: Ticker, Length: 101, dtype: object
101     CMG
102     CMI
103     CMS
104     CNC
105     CNP
       ... 
197      GD
198      GE
199    GILD
200     GIS
201      GL
Name: Ticker, Length: 101, dtype: object
202      GLW
203       GM
204     GOOG
205    GOOGL
206      GPC
       ...  
298      MCD
299     MCHP
300      MCK
301      MCO
302     MDLZ
Name: Ticker, Length: 101, dtype: object
303     MDT
304     MET
305     MGM
306     MHK
307     MKC
       ... 
399     ROK
400     ROL
401     ROP
402    ROST
403     RSG
Name: Ticker, Length: 101, dtype: object
404     RTX
405    SBAC
406    SBUX
407    SCHW
408     SEE
       ... 
500     YUM
501     ZBH
502    ZBRA
503    ZION
504     ZTS
Name: Ticker, Length: 101, dtype: object


In [ ]:





"""
# JavaScript Object Notation
df = pd.read_json(request)
df
# need API key to function!!!
"""